# **Track 2 - Continuous Representation**

In [ ]:
## IMPORTS
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
from gensim.models import FastText
from sklearn.neighbors import NearestNeighbors
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import pandas as pd
import numpy as np
from tqdm import tqdm

## **FastText**

This is the code that should be used to reproduce the results.

In [2]:
## LOAD THE DATA
dev_df = pd.read_csv('dev_responses.csv')
train_df = pd.read_csv('train_responses.csv')

## REMOVE INVALID RESPONSES
train_df = train_df[
    train_df['model_response'].astype(str).str.strip().replace(r'^\W*$', '', regex=True) != ''
].reset_index(drop=True)

## TOKENIZE THE DATA
train_tokens = [str(p).split() for p in train_df['user_prompt']]
dev_tokens = [str(p).split() for p in dev_df['user_prompt']]


In [3]:
## BLEU SCORE CALCULATION
smoother = SmoothingFunction().method3
def evaluate_bleu(dev_df, retrieved_responses):
    scores = []
    for i in range(len(dev_df)):
        ref = str(dev_df.loc[i, 'model_response']).split()
        hyp = str(retrieved_responses[i]).split()
        score = sentence_bleu([ref], hyp, weights=(0.5, 0.5, 0, 0), smoothing_function=smoother)
        scores.append(score)
    return np.mean(scores)

In [7]:
## FASTTEXT WITH OPTIMAL PARAMETERS
model = FastText(
        sentences=train_tokens,
        vector_size=100,
        window=10,
        min_count=1,
        workers=4,
        epochs=100,
        sg=1
    )

In [10]:
## CONVERT TOKENIZED TEXT INTO VECTOR EMBEDDINGS
def get_vec(tokens):
    ''' 
    For each token (word) in tokens, if the word exists in the FastText vocabulary (model.wv), get its vector.
    Then, if any valid vectors were found, return their mean (i.e. sentence embedding). 
    Otherwise, return a zero vector.
    '''
    vecs = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

train_vecs = np.vstack([get_vec(t) for t in train_tokens])
dev_vecs = np.vstack([get_vec(t) for t in dev_tokens])

## NORMALIZE
train_vecs = normalize(train_vecs)
dev_vecs = normalize(dev_vecs)

In [11]:
## FIND MOST SIMILAR PROMPT (using Nearest Neighbors with cosine similarity)
nn = NearestNeighbors(n_neighbors=1, algorithm='brute', metric='cosine')
nn.fit(train_vecs)
_, idxs = nn.kneighbors(dev_vecs)

retrieved_responses = [train_df.iloc[i]['model_response'] for i in idxs.flatten()]
bleu = evaluate_bleu(dev_df, retrieved_responses)

print(f"Average BLEU: {bleu:.4f}")

Average BLEU: 0.0866


### **Hyperparameter Search**

In [ ]:
# define hyperparameter grid
param_grid = [
    {'vector_size': v, 'window': w, 'epochs': e, 'sg': s}
    for v in [100, 300, 512]
    for w in [3, 5, 10]
    for e in [10, 30, 50]
    for s in [0, 1]
]

best_bleu = -1
best_params = None

for params in tqdm(param_grid, desc="Tuning"):
    model = FastText(
        sentences=train_tokens,
        vector_size=params['vector_size'],
        window=params['window'],
        min_count=1,
        workers=4,
        epochs=params['epochs'],
        sg=params['sg']
    )

    def get_vec(tokens):
        vecs = [model.wv[word] for word in tokens if word in model.wv]
        return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

    train_vecs = np.vstack([get_vec(t) for t in train_tokens])
    dev_vecs = np.vstack([get_vec(t) for t in dev_tokens])
    
    train_vecs = normalize(train_vecs)
    dev_vecs = normalize(dev_vecs)

    nn = NearestNeighbors(n_neighbors=1, algorithm='brute', metric='cosine')
    nn.fit(train_vecs)
    _, idxs = nn.kneighbors(dev_vecs)

    retrieved_responses = [train_df.iloc[i]['model_response'] for i in idxs.flatten()]
    bleu = evaluate_bleu(dev_df, retrieved_responses)

    print(f"Params: {params}, BLEU: {bleu:.4f}")
    if bleu > best_bleu:
        best_bleu = bleu
        best_params = params

print(f"\nBest BLEU: {best_bleu:.4f} with params: {best_params}")


Trying 54 combinations...



Tuning:   2%|▏         | 1/54 [00:12<11:04, 12.55s/it]

Params: {'vector_size': 100, 'window': 3, 'epochs': 10, 'sg': 0}, BLEU: 0.0577


Tuning:   4%|▎         | 2/54 [00:25<11:01, 12.73s/it]

Params: {'vector_size': 100, 'window': 3, 'epochs': 10, 'sg': 1}, BLEU: 0.0696


...

Params: {'vector_size': 512, 'window': 10, 'epochs': 30, 'sg': 1}, BLEU: 0.0827


Tuning:  98%|█████████▊| 53/54 [25:43<00:46, 46.01s/it]

Params: {'vector_size': 512, 'window': 10, 'epochs': 50, 'sg': 0}, BLEU: 0.0706


Tuning: 100%|██████████| 54/54 [27:02<00:00, 30.05s/it]

Params: {'vector_size': 512, 'window': 10, 'epochs': 50, 'sg': 1}, BLEU: 0.0855

Best BLEU: 0.0863 with params: {'vector_size': 100, 'window': 10, 'epochs': 50, 'sg': 1}


Note that in the section before we increase the number of epochs to 100 since it improves the result.

### **Create Submission CSV**

In [ ]:
def generate_track2_submission(train_df, dev_df, test_df, output_file='track_2_test.csv'):
    combined_df = pd.concat([train_df, dev_df], ignore_index=True)

    combined_df = combined_df[
        combined_df['model_response'].astype(str).str.strip().replace(r'^\W*$', '', regex=True) != ''
    ].reset_index(drop=True)

    combined_tokens = [str(p).split() for p in combined_df['user_prompt']]
    test_tokens = [str(p).split() for p in test_df['user_prompt']]

    model = FastText(
        sentences=combined_tokens,
        vector_size=100,
        window=10,
        min_count=1,
        workers=4,
        epochs=100,
        sg=1
    )

    def get_vec(tokens):
        vecs = [model.wv[word] for word in tokens if word in model.wv]
        return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

    combined_vecs = np.vstack([get_vec(t) for t in combined_tokens])
    test_vecs = np.vstack([get_vec(t) for t in test_tokens])

    combined_vecs = normalize(combined_vecs)
    test_vecs = normalize(test_vecs)

    nn = NearestNeighbors(n_neighbors=1, algorithm='brute', metric='cosine')
    nn.fit(combined_vecs)
    _, idxs = nn.kneighbors(test_vecs)

    matched_ids = [combined_df.iloc[i]['conversation_id'] for i in idxs.flatten()]
    result_df = pd.DataFrame({
        'conversation_id': test_df['conversation_id'],
        'response_id': matched_ids
    })

    result_df.to_csv(output_file, index=False)
    print(f"Saved Track 2 submission to: {output_file}")

In [13]:
test_df = pd.read_csv('test_prompts.csv')
generate_track2_submission(train_df, dev_df, test_df)

Saved Track 2 submission to: track_2_test.csv
